<a href="https://colab.research.google.com/github/qkrtnwjd4212/Time_Series_Data_Project/blob/main/TA_signal_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.25.2 --force-reinstall

In [ ]:
!pip install pandas_ta yfinance --upgrade

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import pandas_ta as pta

#import pygad

In [ ]:
#gdrive 마운트
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd "/content/gdrive/MyDrive/Colab Notebooks/stock"

/content/gdrive/MyDrive/Colab Notebooks/stock


In [ ]:
# 종목 리스트 - 소비재 섹터 내 10개 대표 종목
tickers = ["HD", "MCD", "LOW", "TJX", "SBUX", "NKE", "MAR", "CMG", "ORLY", "TGT"]

# 결과를 담을 빈 리스트
all_dfs = []

In [ ]:
for ticker in tickers:

  # 2.1 기본 가격 데이터 다운로드
  df = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d", auto_adjust=False,  progress=False)
  df = df[["Open","High","Low","Close","Adj Close","Volume"]].dropna()

  # 2.2 MultiIndex 하나로 평탄화
  if isinstance(df.columns, pd.MultiIndex):
      df.columns = df.columns.get_level_values(0)

  # 2.3 30개 TA 지표 계산
  # — 추세 지표 —
  df.ta.sma(length=20, append=True); df.ta.sma(length=60, append=True)
  df.ta.ema(length=20, append=True); df.ta.ema(length=60, append=True)
  df.ta.wma(length=20, append=True); df.ta.wma(length=60, append=True)
  df.ta.hma(length=20, append=True); df.ta.hma(length=60, append=True)
  df.ta.macd(append=True)
  df.ta.psar(append=True)
  df.ta.tema(length=20, append=True)
  df.ta.adx(append=True)

  # — 모멘텀 지표 —
  df.ta.rsi(length=14, append=True)
  df.ta.stoch(append=True)
  df.ta.roc(length=10, append=True)
  df.ta.cci(length=20, append=True)
  df.ta.mom(length=10, append=True)
  df.ta.trix(length=15, append=True)
  df.ta.cmo(length=14, append=True)
  df.ta.tsi(append=True)

  # — 변동성 지표 —
  df.ta.atr(length=14, append=True)
  df.ta.bbands(length=20, append=True)
  df.ta.kc(append=True)
  df.ta.donchian(append=True)
  df.ta.natr(length=14, append=True)

  # — 거래량 지표 —
  df.ta.obv(append=True)
  df.ta.mfi(length=14, append=True)
  df.ta.ad(append=True)
  df.ta.efi(append=True)
  df.ta.cmf(length=20, append=True)

  # — 기타/종합 지표 —
  df.ta.vwap(append=True)
  df.ta.dpo(length=20, append=True)
  df.ta.pgo(append=True)
  df.ta.bop(append=True)

  # 2.4 Ticker 칼럼 추가
  df["Ticker"] = ticker

  # 2.5 리스트에 저장
  all_dfs.append(df)

# DataFrame으로 합치고 CSV 저장
all_df = pd.concat(all_dfs)
all_df.to_csv("TA_data_30.csv", index=True)

print("✅ 저장 완료: TA_data_30_.csv")

In [ ]:
# 1) CSV 로드
df = pd.read_csv("TA_data_30.csv", index_col=0)
df = df.sort_values(["Ticker", df.index.name])

# 2) 빈 signals DataFrame 생성 (날짜 인덱스만)
signals = pd.DataFrame(index=df.index)

# === 1. 추세(Trend) 지표 ===

# 1-1) SMA(20) 기울기
signals['SMA_signal'] = 0
# 골든크로스 : 단기 MA가 장기 MA를 상향 돌파, 데드크로스 : 단기 MA가 장기 MA를 하향 돌파
signals.loc[(df['SMA_20'].shift(1) <= df['SMA_60'].shift(1)) & (df['SMA_20'] > df['SMA_60']), 'SMA_signal'] = -1 # 매수
signals.loc[(df['SMA_20'].shift(1) >= df['SMA_60'].shift(1)) & (df['SMA_20'] < df['SMA_60']), 'SMA_signal'] = 1 # 매도

# 1-2) EMA(20) 기울기
signals['EMA_signal'] = 0
signals.loc[(df['EMA_20'].shift(1) <= df['EMA_60'].shift(1)) & (df['EMA_20'] > df['EMA_60']), 'EMA_signal'] = -1 # 매수
signals.loc[(df['EMA_20'].shift(1) >= df['EMA_60'].shift(1)) & (df['EMA_20'] < df['EMA_60']), 'EMA_signal'] = 1 # 매도

# 1-3) WMA(20) 기울기
signals['WMA_signal'] = 0
signals.loc[(df['WMA_20'].shift(1) <= df['WMA_60'].shift(1)) & (df['WMA_20'] > df['WMA_60']), 'WMA_signal'] = -1 # 매수
signals.loc[(df['WMA_20'].shift(1) >= df['WMA_60'].shift(1)) & (df['WMA_20'] < df['WMA_60']), 'WMA_signal'] = 1 # 매도

# 1-4) HMA(20) 기울기
signals['HMA_signal'] = 0
signals.loc[(df['HMA_20'].shift(1) <= df['HMA_60'].shift(1)) & (df['HMA_20'] > df['HMA_60']), 'HMA_signal'] = -1 # 매수
signals.loc[(df['HMA_20'].shift(1) >= df['HMA_60'].shift(1)) & (df['HMA_20'] < df['HMA_60']), 'HMA_signal'] = 1 # 매도

# 1-5) TEMA(20) 기울기
signals['TEMA_signal'] = 0
# 가격이 TEMA 위/아래에 위치할 뿐만 아니라 방향성 고려
tema_up = (df["Close"] > df["TEMA_20"]) & (df["TEMA_20"].diff() > 0)
tema_down = (df["Close"] < df["TEMA_20"]) & (df["TEMA_20"].diff() < 0)
signals.loc[tema_up, "TEMA_signal"] = -1
signals.loc[tema_down, "TEMA_signal"] = 1

# 1-6) MACD 교차
signals['MACD_signal'] = 0
# MACD: MACD 라인이 signal선보다 위에 있고, 양수 영역일 때만 매수 시그널
macd_up = (df["MACD_12_26_9"] > df["MACDs_12_26_9"]) & (df["MACD_12_26_9"] > 0)
macd_down = (df["MACD_12_26_9"] < df["MACDs_12_26_9"]) & (df["MACD_12_26_9"] < 0)
signals.loc[macd_up, "MACD_signal"] = -1
signals.loc[macd_down, "MACD_signal"] = 1

# 1-7) PSAR 반전
signals['PSAR_signal'] = 0
# signals.loc[df["Close"] > df["PSARs_0.02_0.2"], "PSAR_signal"] = -1
# signals.loc[df["Close"] < df["PSARl_0.02_0.2"], "PSAR_signal"] = 1

# PSAR 반전 조건을 완화하여 민감도 증가
psar_buy_cond = (df["Close"] > df["PSARs_0.02_0.2"]) & (df["PSARaf_0.02_0.2"] > 0.02)
psar_sell_cond = (df["Close"] < df["PSARl_0.02_0.2"]) & (df["PSARaf_0.02_0.2"] > 0.02)
signals.loc[psar_buy_cond, "PSAR_signal"] = -1
signals.loc[psar_sell_cond, "PSAR_signal"] = 1

# 1-8) ADX 시스템 (+DI vs -DI)
signals['DI_signal'] = 0
signals.loc[df['DMP_14'] > df['DMN_14'], 'DI_signal'] = -1
signals.loc[df['DMP_14'] < df['DMN_14'], 'DI_signal'] = 1


# === 2. 모멘텀(Momentum) 지표 ===

# 2-1) RSI
signals['RSI_signal'] = 0
# 매수 조건: 과매도 + 최근 하락 추세 반전
buy_cond = (df['RSI_14'] < 25) & (df['RSI_14'] > df['RSI_14'].shift(1))
signals.loc[buy_cond, 'RSI_signal'] = -1
# 매도 조건: 과매수 + RSI 하락 반전
sell_cond = (df['RSI_14'] > 75) & (df['RSI_14'] < df['RSI_14'].shift(1))
signals.loc[sell_cond, 'RSI_signal'] = 1

# 2-2) Stochastic:  과매도 구간에서 %K가 %D 상향 돌파할 때만 매수
signals['STOCH_signal'] = 0
signals.loc[
    (df['STOCHk_14_3_3'] > df['STOCHd_14_3_3']) & (df['STOCHk_14_3_3'].shift(1) <= df['STOCHd_14_3_3'].shift(1)),
    'STOCH_signal'] = -1
signals.loc[
    (df['STOCHk_14_3_3'] < df['STOCHd_14_3_3']) & (df['STOCHk_14_3_3'].shift(1) >= df['STOCHd_14_3_3'].shift(1)),
    'STOCH_signal'] = 1

# 2-3) ROC_10 (부호로만 판단)
signals['ROC_signal'] = 0
signals.loc[df['ROC_10'] > 0.01, 'ROC_signal'] = -1 # ROC > 0 (강세 모멘텀) -> 매수
signals.loc[df['ROC_10'] < -0.01, 'ROC_signal'] = 1 # ROC < 0 (약세 모멘텀) -> 매도

# 2-4) CCI
signals['CCI_signal'] = 0
signals.loc[df['CCI_20_0.015'] < -100, 'CCI_signal'] = -1
signals.loc[df['CCI_20_0.015'] > 100, 'CCI_signal'] = 1

# 2-5) MOM
signals['MOM_signal'] = 0
signals.loc[df['MOM_10'] > 0, 'MOM_signal'] = -1 # MOM > 0 (강세 모멘텀) -> 매수
signals.loc[df['MOM_10'] < 0, 'MOM_signal'] = 1 # MOM < 0 (약세 모멘텀) -> 매도

# 2-6) TRIX 교차
signals['TRIX_signal'] = 0
signals.loc[df['TRIX_15_9'] > df['TRIXs_15_9'], 'TRIX_signal'] = -1
signals.loc[df['TRIX_15_9'] < df['TRIXs_15_9'], 'TRIX_signal'] = 1

# 2-7) CMO
signals['CMO_signal'] = 0
signals.loc[df['CMO_14'] < -50, 'CMO_signal'] = -1
signals.loc[df['CMO_14'] > 50,  'CMO_signal'] = 1

# 2-8) TSI vs Signal
signals['TSI_signal'] = 0
signals.loc[df['TSI_13_25_13'] > df['TSIs_13_25_13'], 'TSI_signal'] = -1
signals.loc[df['TSI_13_25_13'] < df['TSIs_13_25_13'], 'TSI_signal'] = 1


# === 3. 변동성(Volatility) 지표 ===

# 3-1) ATR Ratio 변화
signals['ATRr_signal'] = 0
delta_atr = df['ATRr_14'].diff()
signals.loc[delta_atr > 0.2, 'ATRr_signal'] = 1
signals.loc[delta_atr < -0.2, 'ATRr_signal'] = -1

# 3-2) Bolinget Band
signals["BB_signal"] = 0
# 매수 시그널: 가격이 하단 근처(0.1 이하) + 밴드가 확장되고 있음 (변동성 증가 → 상승 가능성)
buy_cond = (df["BBP_20_2.0"] < 0.1) & (df["BBB_20_2.0"].diff() > 0)
signals.loc[buy_cond, "BB_signal"] = -1

# 매도 시그널: 가격이 상단 근처(0.9 이상) + 밴드가 축소되고 있음 (상승 피로 → 하락 가능성)
sell_cond = (df["BBP_20_2.0"] > 0.9) & (df["BBB_20_2.0"].diff() < 0)
signals.loc[sell_cond, "BB_signal"] = 1

# 3-4) Keltner Channel 돌파
signals['KC_signal'] = 0
signals.loc[df['Close'] > df['KCUe_20_2'], 'KC_signal'] = -1
signals.loc[df['Close'] < df['KCLe_20_2'], 'KC_signal'] = 1

# 3-5) Donchian Channel 돌파
signals['DC_signal'] = 0
# signals.loc[df['Close'] > df['DCU_20_20'], 'DC_signal'] = -1
# signals.loc[df['Close'] < df['DCL_20_20'], 'DC_signal'] = 1

# Donchian Channel 돌파 시 중간 값(DCM)도 반영하여 매매 기회 증가
dc_buy_cond = (df['Close'] > df['DCU_20_20']) | ((df['Close'] > df['DCM_20_20']) & (df['Close'].shift(1) < df['DCM_20_20']))
dc_sell_cond = (df['Close'] < df['DCL_20_20']) | ((df['Close'] < df['DCM_20_20']) & (df['Close'].shift(1) > df['DCM_20_20']))
signals.loc[dc_buy_cond, 'DC_signal'] = -1
signals.loc[dc_sell_cond, 'DC_signal'] = 1

# 3-6) NATR 변화
signals['NATR_signal'] = 0
# signals.loc[df['NATR_14'] > df['NATR_14'].shift(1), 'NATR_signal'] = 1
# signals.loc[df['NATR_14'] < df['NATR_14'].shift(1), 'NATR_signal'] = -1
signals.loc[df['NATR_14'].diff() > 0.05, 'NATR_signal'] = 1
signals.loc[df['NATR_14'].diff() < -0.05, 'NATR_signal'] = -1

# === 4. 거래량(Volume) 지표 ===

# 4-1) OBV 변화
signals['OBV_signal'] = 0
delta_obv = df['OBV'].diff()
signals.loc[delta_obv > 1e6, 'OBV_signal'] = -1
signals.loc[delta_obv < -1e6, 'OBV_signal'] = 1

# 4-2) MFI
signals['MFI_signal'] = 0
signals.loc[df['MFI_14'] < 20, 'MFI_signal'] = -1
signals.loc[df['MFI_14'] > 80, 'MFI_signal'] = 1

# 4-3) AD (A/D Line) 변화
signals['AD_signal'] = 0
delta_ad = df['AD'].diff()
signals.loc[delta_ad > 1e6, 'AD_signal'] = -1
signals.loc[delta_ad < -1e6, 'AD_signal'] = 1

# 4-4) EFI 변화
signals['EFI_signal'] = 0
delta_efi = df['EFI_13'].diff()
signals.loc[delta_efi > 1000000, 'EFI_signal'] = -1
signals.loc[delta_efi < -1000000, 'EFI_signal'] = 1

# 4-5) CMF
signals['CMF_signal'] = 0
signals.loc[df['CMF_20'] > 0.2, 'CMF_signal'] = -1
signals.loc[df['CMF_20'] < -0.2, 'CMF_signal'] = 1


# === 5. 기타/종합(Misc) 지표 ===

# 5-1) VWAP_D 돌파
signals['VWAP_signal'] = 0
# signals.loc[df['Close'] > df['VWAP_D'], 'VWAP_signal'] = -1
# signals.loc[df['Close'] < df['VWAP_D'], 'VWAP_signal'] = 1
signals.loc[(df['Close'] > df['VWAP_D'] * 1.01), 'VWAP_signal'] = -1
signals.loc[(df['Close'] < df['VWAP_D'] * 0.99), 'VWAP_signal'] = 1

# 5-2) DPO
signals['DPO_signal'] = 0
# signals.loc[df['DPO_20'] > 0, 'DPO_signal'] = -1
# signals.loc[df['DPO_20'] < 0, 'DPO_signal'] = 1
signals.loc[df['DPO_20'] > 0.05, 'DPO_signal'] = -1
signals.loc[df['DPO_20'] < -0.05, 'DPO_signal'] = 1

# 5-3) PGO
signals['PGO_signal'] = 0
# signals.loc[df['PGO_14'] > df['PGO_14'].shift(1), 'PGO_signal'] = -1
# signals.loc[df['PGO_14'] < df['PGO_14'].shift(1), 'PGO_signal'] = 1
signals.loc[df['PGO_14'].diff() > 0.02, 'PGO_signal'] = -1
signals.loc[df['PGO_14'].diff() < -0.02, 'PGO_signal'] = 1

# 5-4) BOP
signals['BOP_signal'] = 0
# signals.loc[df['BOP'] > 0, 'BOP_signal'] = -1
# signals.loc[df['BOP'] < 0, 'BOP_signal'] = 1
signals.loc[df['BOP'] > 0.05, 'BOP_signal'] = -1
signals.loc[df['BOP'] < -0.05, 'BOP_signal'] = 1

signals.fillna(0, inplace=True) # NaN을 0으로 채움

# --- 원본 Adj Close 값과 Ticker도 추가 ---
signals["Adj Close"] = df["Adj Close"]
signals["Ticker"] = df["Ticker"]

# 3) CSV 저장
signals.to_csv("All_TA_signals.csv", index=True)
print("✅ 저장 완료: All_TA_signals.csv")

✅ 저장 완료: All_TA_signals.csv


In [ ]:
# 1. 시그널 파일 불러오기
df = pd.read_csv("All_TA_signals.csv")

# 2. 시그널 컬럼만 추출
signal_cols = [col for col in df.columns if col.endswith('_signal')]

# 3. 시그널 카운트 및 빈도 계산
counts = []

for col in signal_cols:
    signal_counts = df[col].value_counts().to_dict()
    buy = signal_counts.get(-1, 0)
    hold = signal_counts.get(0, 0)
    sell = signal_counts.get(1, 0)
    total = buy + hold + sell
    trade_freq = (buy + sell) / total if total > 0 else 0
    counts.append({
        "Signal": col,
        "Buy (-1)": buy,
        "Hold (0)": hold,
        "Sell (1)": sell,
        "Trade Frequency": round(trade_freq, 4)
    })

# 4. 결과 저장
summary_df = pd.DataFrame(counts)
summary_df.to_csv("TA_signal_counts_2.csv", index=False)

print("✅ 시그널 카운트 및 트레이드 빈도 저장 완료: TA_signal_counts.csv")

✅ 시그널 카운트 및 트레이드 빈도 저장 완료: TA_signal_counts.csv


In [ ]:
print(summary_df)

          Signal  Buy (-1)  Hold (0)  Sell (1)  Trade Frequency
0     SMA_signal        66      7390        64           0.0173
1     EMA_signal        60      7401        59           0.0158
2     WMA_signal        85      7350        85           0.0226
3     HMA_signal       187      7141       192           0.0504
4    TEMA_signal      2932      1863      2725           0.7523
5    MACD_signal      2392      3219      1909           0.5719
6    PSAR_signal         0      7520         0           0.0000
7      DI_signal      3898       140      3482           0.9814
8     RSI_signal        24      7466        30           0.0072
9   STOCH_signal       897      5724       899           0.2388
10    ROC_signal      4007       109      3404           0.9855
11    CCI_signal      1360      4421      1739           0.4121
12    MOM_signal      4010       103      3407           0.9863
13   TRIX_signal      3575       230      3715           0.9694
14    CMO_signal       120      7206    